# Préparation des données

In [8]:
from sklearn.preprocessing import OneHotEncoder
import pandas as pd
from csv import reader

data_base = pd.read_csv('../Vessel_Total_Clean_Final.csv')

encoder = OneHotEncoder()
data_encoded = encoder.fit_transform(data_base[['vessel_type']])
variables_used = data_base[['sog','cog','lat','lon']]

# Apprentissage non-supervisé

In [ ]:
from sklearn.cluster import KMeans, DBSCAN, AgglomerativeClustering

# Prédiction avec KMeans :
kmeans = KMeans(n_clusters=5, random_state=42)
clusters_kmeans = kmeans.fit_predict(variables_used)

# Prédiction avec DBSCAN : 
dbscan = DBSCAN(eps=0.5, min_samples=5)
clusters_dbscan = dbscan.fit_predict(variables_used)

# Prédiction avec Agglomative : 
agglo = AgglomerativeClustering(n_clusters=5, linkage='ward')
clusters_agglo = agglo.fit_predict(variables_used)




# Métriques pour l'apprentissage non-supervisé

In [ ]:
from sklearn.metrics import silhouette_score, calinski_harabasz_score, davies_bouldin_score
print(silhouette_score(variables_used, clusters))
print(calinski_harabasz_score(variables_used, clusters))
print(davies_bouldin_score(variables_used, clusters))

SyntaxError: invalid syntax. Perhaps you forgot a comma? (3425351733.py, line 2)

# Visualisation sur un carte

In [ ]:
import plotly.express as px

# Visualisation des résultats DBSCAN
fig_db = px.scatter_mapbox(data_base, lat="lat", lon="lon",
                           mapbox_style="open-street-map", zoom=3,
                           title="Clusters DBSCAN")
fig_db.show()

# Visualisation des résultats Agglomerative
fig_agglo = px.scatter_mapbox(data_base, lat="lat", lon="lon",
                              mapbox_style="open-street-map", zoom=3,
                              title="Clusters Agglomerative")
fig_agglo.show()

C:\Users\alban\AppData\Local\Temp\ipykernel_4068\3679573020.py:4: DeprecationWarning: *scatter_mapbox* is deprecated! Use *scatter_map* instead. Learn more at: https://plotly.com/python/mapbox-to-maplibre/
  fig_db = px.scatter_mapbox(data_base, lat="lat", lon="lon", color="green",


ValueError: Value of 'color' is not the name of a column in 'data_frame'. Expected one of ['id', 'mmsi', 'base_date_time', 'lat', 'lon', 'sog', 'cog', 'heading', 'vessel_name', 'imo', 'call_sign', 'vessel_type', 'status', 'length', 'width', 'draft', 'cargo', 'transceiver_class'] but received: green